In [2]:
!git clone https://github.com/danielgatis/rembg.git

fatal: destination path 'rembg' already exists and is not an empty directory.


In [4]:
from pathlib import Path
from rembg import remove, new_session

session = new_session()

for file in Path('path/to/folder').glob('*.png'):
    input_path = str(file)
    output_path = str(file.parent / (file.stem + ".out.png"))

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input = i.read()
            output = remove(input, session=session)
            o.write(output)

# display_image()

In [67]:
import cv2
import os

def video_to_frames(video_path, output_folder="frames", fps=30):
    """
    Converts a video into frames and saves them to the specified folder.

    Parameters:
        video_path (str): Path to the input video file.
        output_folder (str): Directory to save the extracted frames. Default is 'frames'.
        fps (int): Frames per second to extract. Default is 30.
    """
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)

    # Check if the video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' does not exist.")
        return

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    original_fps = video_capture.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps / fps) if fps <= original_fps else 1

    frame_count = 0
    saved_count = 1

    while True:
        success, frame = video_capture.read()
        if not success:
            break

        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count}.png")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    total_img = saved_count - 1
    video_capture.release()
    print(f"Extracted {total_img} frames and saved to '{output_folder}'")

# Example usage
# video_to_frames("input_video.mp4", fps=10)

In [68]:
video_to_frames("input_video.mp4", fps=30)

Extracted 102 frames and saved to 'frames'


In [51]:
import cv2
import os

def video_to_frames_with_text_and_fps(video_path, output_folder="frames", fps=30, text=None, font_color=(255, 0, 0), text_position=(50, 50), font_scale=1, thickness=2, padding=(0, 0)):
    """
    Converts a video into frames, optionally adds text to each frame, and saves them to separate folders:
    one for frames with text (outside the 'frames' folder) and one for frames without text (inside the 'frames' folder).

    Parameters:
        video_path (str): Path to the input video file.
        output_folder (str): Directory to save the extracted frames. Default is 'frames'.
        fps (int): Frames per second to extract. Default is 30.
        text (str): Optional text to overlay on each frame. If None, no text is added.
        font_color (tuple): Font color in BGR format. Default is (255, 0, 0) (blue).
        text_position (tuple): Position of the text (x, y). Default is (50, 50).
        font_scale (float): Font size. Default is 1.
        thickness (int): Thickness of the text. Default is 2.
        padding (tuple): Padding around the text as (x_padding, y_padding). Default is (0, 0).
    """
    # Ensure the output directories exist
    frames_with_text_folder = "frames_with_text"
    frames_without_text_folder = os.path.join(output_folder)

    os.makedirs(frames_with_text_folder, exist_ok=True)
    os.makedirs(frames_without_text_folder, exist_ok=True)

    # Check if the video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' does not exist.")
        return

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    original_fps = video_capture.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps / fps) if fps <= original_fps else 1

    frame_count = 0
    saved_with_text_count = 1
    saved_without_text_count = 1

    while True:
        success, frame = video_capture.read()
        if not success:
            break

        if frame_count % frame_interval == 0:
            # Save the frame without text in the 'frames' folder
            frame_filename_without_text = os.path.join(frames_without_text_folder, f"frame_{saved_without_text_count}.png")
            cv2.imwrite(frame_filename_without_text, frame)
            saved_without_text_count += 1

            # If text is provided, add it to the frame and save to the 'frames_with_text' folder
            if text:
                font = cv2.FONT_HERSHEY_SIMPLEX

                # Add the text to the frame with padding
                text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                x, y = text_position
                x += padding[0]
                y += padding[1]
                bottom_left_corner = (x, y + text_size[1])
                cv2.putText(frame, text, bottom_left_corner, font, font_scale, font_color, thickness)

                # Save the frame with text in the 'frames_with_text' folder
                frame_filename_with_text = os.path.join(frames_with_text_folder, f"frame_{saved_with_text_count}.png")
                cv2.imwrite(frame_filename_with_text, frame)
                saved_with_text_count += 1

        frame_count += 1

    total_with_text = saved_with_text_count - 1
    total_without_text = saved_without_text_count - 1
    video_capture.release()

    print(f"Extracted {total_without_text} frames without text and saved to '{frames_without_text_folder}'")
    print(f"Extracted {total_with_text} frames with text and saved to '{frames_with_text_folder}'")

# Example usage
# video_to_frames_with_text_and_fps("input_video.mp4", fps=10, text="This is a test", font_color=(0, 255, 0), text_position=(100, 100), font_scale=2, thickness=3, padding=(10, 20))


In [75]:
video_to_frames_with_text_and_fps("input_video.mp4", fps=30, text="hello", font_color=(0, 255, 0), text_position=(0, 0), font_scale=25, thickness=30, padding=(0, 0))


Extracted 102 frames without text and saved to 'frames'
Extracted 102 frames with text and saved to 'frames_with_text'


In [57]:
from pathlib import Path
from rembg import remove, new_session
import os

def remove_background_from_frames(input_folder, output_folder):
    """
    Removes the background from all PNG images in the specified input folder
    and saves the output images to the specified output folder with 'frame_bgr' naming.

    Parameters:
        input_folder (str): Path to the folder containing input PNG images (frames).
        output_folder (str): Path to the folder where output images will be saved (frames_bgr).
    """
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Create a new session for background removal
    session = new_session()

    # Iterate over all PNG files in the input folder
    for file in sorted(Path(input_folder).glob('frame_*.png'), key=lambda x: int(x.stem.split('_')[1])):
        input_path = str(file)
        output_path = str(Path(output_folder) / (file.stem + ".png"))

        # Read input image, remove background, and save the result
        with open(input_path, 'rb') as i:
            with open(output_path, 'wb') as o:
                input_data = i.read()
                output_data = remove(input_data, session=session)
                o.write(output_data)

    print(f"Background removal completed for all frames in '{input_folder}' and saved to '{output_folder}'.")

# Example usage
# remove_background_from_frames("frames", "frames_bgr")


In [76]:
remove_background_from_frames("frames", "frames_bgr")

Background removal completed for all frames in 'frames' and saved to 'frames_bgr'.


In [93]:
import cv2
import os

def merge_images_from_folders(folder1, folder2, output_folder="merged_frames", opacity=1):
    """
    Merges images from two folders with the same filenames, overlays images from folder2 on top of images from folder1,
    and saves them to the output folder. Allows setting the opacity of the overlay image.

    Parameters:
        folder1 (str): Path to the first folder (base images).
        folder2 (str): Path to the second folder (overlay images with transparency).
        output_folder (str): Directory to save the merged images. Default is 'merged_frames'.
        opacity (float): Opacity of the overlay image (0.0 to 1.0). Default is 0.5.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of images in both folders
    images_folder1 = sorted(os.listdir(folder1))
    images_folder2 = sorted(os.listdir(folder2))

    if len(images_folder1) != len(images_folder2):
        print("Error: The number of images in both folders is not the same.")
        return

    # Iterate through images in both folders
    for img1_name, img2_name in zip(images_folder1, images_folder2):
        # Check if the filenames are the same
        if img1_name != img2_name:
            print(f"Skipping files with different names: {img1_name} and {img2_name}")
            continue
        
        # Read the images (with transparency for img2)
        img1 = cv2.imread(os.path.join(folder1, img1_name), cv2.IMREAD_UNCHANGED)
        img2 = cv2.imread(os.path.join(folder2, img2_name), cv2.IMREAD_UNCHANGED)

        if img1 is None or img2 is None:
            print(f"Error: One of the images could not be read: {img1_name} or {img2_name}")
            continue

        # Ensure both images have 4 channels (RGBA) for transparency handling
        if img1.shape[2] != 4:
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2BGRA)
        if img2.shape[2] != 4:
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2BGRA)

        # Resize img2 to match img1's size if they are not the same size
        if img1.shape[:2] != img2.shape[:2]:
            img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

        # Separate the alpha channel for blending
        alpha = img2[:, :, 3] / 255.0 * opacity
        alpha_inv = 1.0 - alpha

        # Blend the two images
        for c in range(3):  # Blend RGB channels
            img1[:, :, c] = (alpha_inv * img1[:, :, c] + alpha * img2[:, :, c]).astype('uint8')

        # Save the merged image to the output folder
        output_filename = os.path.join(output_folder, img1_name)
        cv2.imwrite(output_filename, img1)

    print(f"Images merged and saved to '{output_folder}'.")

# Example usage
# merge_images_from_folders("frames/with_text", "frames/without_text", "merged_frames", opacity=0.7)

In [94]:
merge_images_from_folders("frames_with_text", "frames_bgr", "merged_frames",opacity= 1)

Images merged and saved to 'merged_frames'.


In [36]:
import cv2
import os

def frames_to_video(input_folder, output_video_path, fps=30):
    """
    Creates a video from a sequence of images in a folder.

    Parameters:
        input_folder (str): Directory containing the input frames (images).
        output_video_path (str): Path where the output video will be saved.
        fps (int): Frames per second for the output video. Default is 30.
    """
    # Get the list of all image files in the folder, sorted by filename
    images = [img for img in os.listdir(input_folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sorting by frame number (assuming names like frame_1.png)

    # Check if there are any images
    if not images:
        print(f"Error: No images found in '{input_folder}'.")
        return

    # Read the first image to get the dimensions
    first_image = cv2.imread(os.path.join(input_folder, images[0]))
    height, width, _ = first_image.shape

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using the mp4 codec
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through all images and add them to the video
    for image_name in images:
        image_path = os.path.join(input_folder, image_name)
        frame = cv2.imread(image_path)
        video_writer.write(frame)

    # Release the VideoWriter object
    video_writer.release()

    print(f"Video saved to '{output_video_path}'")

# Example usage
# frames_to_video("frames", "output_video.mp4", fps=10)


In [95]:
frames_to_video("merged_frames", "output_video.mp4", fps=30)

Video saved to 'output_video.mp4'


In [96]:
import os
import shutil

# Remove directories and their contents
shutil.rmtree('merged_frames', ignore_errors=True)
shutil.rmtree('frames', ignore_errors=True)
shutil.rmtree('frames_bgr', ignore_errors=True)
shutil.rmtree('frames_with_text', ignore_errors=True)

# Create directories if they don't exist
os.makedirs('merged_frames', exist_ok=True)
os.makedirs('frames', exist_ok=True)
os.makedirs('frames_bgr', exist_ok=True)
os.makedirs('frames_with_text', exist_ok=True)